## Imports 

In [13]:
import os
import sys
import pickle
import joblib
import argparse
from tqdm import tqdm
from os import path
from git import Repo
from os.path import exists
from os import mkdir, remove, rename
# from .autonotebook import tqdm as notebook_tqdm

import math
import random
import numpy as np
from sklearn.preprocessing import RobustScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.dataset import Dataset, random_split

torch.manual_seed(42)

# set up train device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device: {}".format(device))

# setup project root dir
COLAB = 'google.colab' in sys.modules
if COLAB:
  root_dir = '/content'
  %mkdir ./data/
else:
  repo = Repo(".", search_parent_directories=True)
  root_dir = repo.git.rev_parse("--show-toplevel")
  sys.path.insert(0, root_dir+'/models/')
print("root: {}".format(root_dir))



device: cpu
root: /home/leeoos/Projects/master/airo-rl/acrobatic-agents


In [15]:
# Import data and build dataloader
dataset_len = 6000
raw_expert_observations = np.load(root_dir+'/data/expert-observations-'+str(dataset_len)+'.npy', allow_pickle=True)
expert_actions = np.load(root_dir+'/data/expert-actions-'+str(dataset_len)+'.npy', allow_pickle=True)

In [16]:
# Robust scaling

# Create a robust scaler object
scaler = RobustScaler()

# Fit the scaler to your data
scaler.fit(raw_expert_observations)
expert_observations = scaler.transform(raw_expert_observations)

# This will save the scaler to a file named 'scaler.joblib'
joblib.dump(scaler, root_dir+'/data/scaler-'+str(dataset_len)+'.joblib')  

# with open(root_dir+'/data/scaler-'+str(dataset_len)+'.joblib', 'wb') as file:
#     pickle.dump(scaler, file)


['/home/leeoos/Projects/master/airo-rl/acrobatic-agents/data/scaler-6000.joblib']

## Data

In [17]:
# Dataset class

class ExpertDataSet(Dataset):

    def __init__(self, expert_observations, expert_actions):
        self.observations = torch.from_numpy(expert_observations).float()
        self.actions = self.__preprocess__(torch.from_numpy(expert_actions))
        

    def __getitem__(self, idx):
        # return (self.observations[index], self.actions[index])
        # normalized_observations = 2 * ((self.observations[idx] - self.observations.min()) / (self.observations.max() - self.observations.min())) - 1
        # normalized_actions = 2 * ((self.actions[idx] - self.actions.min()) / (self.actions.max() - self.actions.min())) - 1
        # normalized_data = (normalized_observations, self.actions[idx])
        # return normalized_data
        return self.observations[idx], self.actions[idx]


    def __len__(self):
        return len(self.observations)
    
    
    def __preprocess__(self, data, clip_value=1e38):
        # Clip values to a maximum and minimum range
        data = torch.clamp(data, min=-clip_value, max=clip_value)
        
        # Convert to float
        return data.float()
    
    def __min__max__(self):
        return self.observations.min(), self.observations.max() 

In [18]:
# Make Datasets 

count_discarded_numpy = 0
count_discarded = 0

new_exp_action = expert_actions

list_of_index_to_drop = []
for i, a in enumerate(expert_actions):
  if (a > 1e2).any() or (a > 1e2).any():
  # if not np.isfinite(a).all(): 
    list_of_index_to_drop.append(i)
    # print(i)
    # print(a)
    count_discarded_numpy+=1
    # break


print("Expert actions len: {}".format(len(expert_actions)))
print("Expert observations len: {}".format(len(expert_observations)))

expert_dataset = ExpertDataSet(expert_observations, expert_actions)

for i in range(len(expert_dataset)):
  a = expert_dataset.__getitem__(i)[1]
  # print(a.max())
  # print(a.min())
  if (a > 1e2).any() or (a < -1e2).any() :
  # if not torch.isfinite(a).any():
    count_discarded += 1
    # print(a)

print("Discarded data:")
print("Discarded form np: {}".format(count_discarded_numpy))
print("Discarded form torch: {}".format(count_discarded))

min_val, max_val = expert_dataset.__min__max__()

print("Statistics: ")
print("Observations min: {}".format(min_val))
print("Observations max: {}".format(max_val))


Expert actions len: 6000
Expert observations len: 6000
Discarded data:
Discarded form np: 0
Discarded form torch: 0
Statistics: 
Observations min: -51.51035690307617
Observations max: 32.14270782470703


In [19]:
# check correct scaling
idx = random.randint(0, dataset_len)
print(raw_expert_observations[idx])
print(raw_expert_observations.min())
print(raw_expert_observations.max())
print(expert_dataset.__getitem__(idx)[0])
print(expert_observations.min())
print(expert_observations.max())


[ 7.74428571e-01  7.42610991e-01  5.56678772e-02  3.60411406e-02
  2.24084258e-02  8.66851151e-01  1.42497122e-01 -7.74979517e-02
 -4.71442193e-01  2.99518347e-01  1.09467924e-01  1.17693186e-01
  6.67220414e-01  1.92319885e-01 -6.57037348e-02 -7.16598213e-01
  5.39390802e-01  9.99951363e-03  2.01223373e-01  4.93032038e-01
  1.83307692e-01 -5.20066731e-02 -8.48889291e-01  1.02375031e-01
 -1.70117855e-01  1.45956576e-01  9.21937287e-01 -1.17027409e-01
 -1.41107887e-01  3.41210723e-01  1.40183806e-01 -5.05826250e-01
  2.02743530e-01  9.56451058e-01 -1.91668160e-02 -1.82316944e-01
 -2.27144405e-01  9.18982029e-02 -7.16203172e-01  2.01081216e-01
  9.96377587e-01 -1.00418376e-02 -8.44366029e-02  1.13360060e-03
  2.34437704e-01  1.56937838e-02  3.04082155e-01  8.66921604e-01
  1.87656313e-01  1.29851401e-01 -4.43137348e-01  6.21194839e-02
 -1.50321901e-01  2.33774662e-01  9.39353943e-01  2.10290700e-01
  8.86233225e-02 -2.56003708e-01 -9.88674164e-03 -2.58767217e-01
  1.85185075e-01  9.39353

In [20]:
# Data Loaders

batch_size = 128
train_prop = 0.7
train_size = int(train_prop * len(expert_dataset))
test_size = int(0.2 * len(expert_dataset))
val_size = int(0.1 * len(expert_dataset))
train_expert_dataset, test_expert_dataset,val_expert_dataset = random_split(expert_dataset, [train_size, test_size,val_size])

train_loader = torch.utils.data.DataLoader(dataset=train_expert_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_expert_dataset, batch_size=batch_size, shuffle=False)
val_loader = torch.utils.data.DataLoader( dataset=val_expert_dataset, batch_size=batch_size, shuffle=False)


print("Shapes:")
print("actions shape: {}".format(train_loader.dataset.__getitem__(0)[1].shape))
print("observations shape: {}".format(train_loader.dataset.__getitem__(0)[0].shape))


Shapes:
actions shape: torch.Size([36])
observations shape: torch.Size([197])


# BCO

## Model

In [43]:
# Policy Agent
import bco_agents as bco

obs_space = train_loader.dataset.__getitem__(0)[0].shape[0]
action_space = train_loader.dataset.__getitem__(0)[1].shape[0]
policy = bco.BCOAgentFC(obs_space, action_space, h_size=obs_space*2).to(device)

print("Policy net: {}".format(policy))

Policy net: BCOAgentFC(
  (fc1): Linear(in_features=197, out_features=394, bias=True)
  (bn1): BatchNorm1d(394, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (fc2): Linear(in_features=394, out_features=36, bias=True)
)


In [27]:
# Policy Agent

import bco_cnn as bco_cnn
obs_space = train_loader.dataset.__getitem__(0)[0].shape[0]
action_space = train_loader.dataset.__getitem__(0)[1].shape[0]
policy = bco_cnn.BCO_cnn(obs_space, action_space).to(device) # fix add hidden layer size

print("Policy net: {}".format(policy))

Policy net: BCO_cnn(
  (conv1): Conv1d(1, 36, kernel_size=(5,), stride=(2,), padding=(1,))
  (conv2): Conv1d(36, 36, kernel_size=(3,), stride=(2,), padding=(2,))
  (fc1): Linear(in_features=1800, out_features=72, bias=True)
  (fc2): Linear(in_features=72, out_features=36, bias=True)
  (LRelu): LeakyReLU(negative_slope=0.01)
)


In [12]:
# Policy Agent

import encoder as en
obs_space = train_loader.dataset.__getitem__(0)[0].shape[0]
action_space = train_loader.dataset.__getitem__(0)[1].shape[0]
policy = en.Encoder(obs_space, action_space, 50).to(device) # fix add hidden layer size

print("Policy net: {}".format(policy))

Policy net: Encoder(
  (conv1): Conv1d(1, 36, kernel_size=(5,), stride=(2,), padding=(1,))
  (conv2): Conv1d(36, 36, kernel_size=(3,), stride=(2,), padding=(2,))
  (fc1): Linear(in_features=1800, out_features=72, bias=True)
  (fc2): Linear(in_features=72, out_features=36, bias=True)
  (LRelu): LeakyReLU(negative_slope=0.01)
)


## Training

In [21]:
# Train functions

def train(
        policy,
        train_epochs,
        train_loader, 
        val_loader,
        optimizer,
        loss_criterion,
        scheduler,
        thrashold
    ):

    policy.train()
    loss = 0
    epoch_loss = 0
    unused_val = 0

    with tqdm(total=train_epochs, leave=True) as pbar:
        for epoch in range(train_epochs):
            for batch_idx, (data, target) in enumerate(train_loader):

                obs, expert_action = data.to(device), target.to(device)
                obs = obs.float()

                if policy.name == 'bco-cnn':
                    obs = obs.unsqueeze(1)
                

                optimizer.zero_grad()

                student_action = policy(obs)
                expert_action = expert_action.float()

                loss = loss_criterion(student_action, expert_action)
                # loss.register_hook(lambda grad: print(grad))
                loss.backward()
                # print("Loss: {}".format(loss.item()))

                if not loss.item() == torch.inf: 
                    epoch_loss += loss.item()
                    optimizer.step()
                    

                else:
                    unused_val += 1
                    print("### BATCH {} ###".format(batch_idx))
                    print(f'obs -> {obs}')
                    print("\n______________________________________________________________________________")
                    print(f'expert_action -> {expert_action}')
                    print("\n______________________________________________________________________________")
                    print(f'student_action -> {student_action}')
                    print("\n______________________________________________________________________________")
                    return expert_action,student_action

                res = print_gradients(policy)
                
                if torch.isnan(student_action).any(): 
                    print('e successo')
                    break

                if res == 1: 
                    print("\n______________________________________________________________________________")
                    print(student_action.shape)
                    for i, ea in enumerate(expert_action):
                        if not np.isfinite(ea).all():
                            print(i+64)
                            print(f'expert_action -> {ea}')

                    print("\n______________________________________________________________________________")
                    print(f'Max expert_action -> {expert_action.max()}')
                    print(f'Min expert_action -> {expert_action.min()}')
                    print(f'Max student_action -> {student_action.max()}')
                    print(f'Min student_action -> {student_action.min()}')
                    break
                
            # deactivate scheduler
            # if epoch % 50 == 0 and epoch < thrashold :
            #     scheduler.step()
            
            # compute accuracy
            # print("Epoch {}".format(epoch))
            # print("Train Loss: {}".format(epoch_loss/(batch_idx+1)))
            # validation(test_loader,policy,loss_criterion,num_epochs=eval_epochs)
            # print("Unused Loss: {}".format(unused_val))
            t_loss = epoch_loss/(batch_idx+1)
            v_loss = validation(val_loader, policy, loss_criterion)
            epoch_loss = 0
            unused_val = 0
            pbar.set_postfix(train=t_loss, validation=v_loss)
            pbar.update(1)
            
        
        print("###############################################################################\n")
        print("Train Loss: {}".format(t_loss))
        print("Validation Loss: {}".format(v_loss))
        print("###############################################################################\n")


def validation(loader, policy,loss_criterion):
    policy.eval()
    epoch_loss = 0
    for batch_idx, (data, target) in enumerate(loader):
        obs, expert_action = data.to(device), target.to(device)
        obs = obs.float()
        if policy.name == 'bco-cnn':
            obs = obs.unsqueeze(1)
        student_action = policy(obs)
        loss = loss_criterion(student_action, expert_action)
        epoch_loss += loss.item()

    return epoch_loss/(batch_idx+1)

def print_gradients(policy):
    for name, param in policy.named_parameters():
        if param.requires_grad:
            if torch.isnan(param.grad).any(): 
                return 1#break
            # print(f"Gradient of {name}: {param.grad}")
    return 0

In [24]:
# Train module

loss_criterion = nn.MSELoss()
# Create a learning rate scheduler
step_size = 80
gamma = 0.3
optimizer =  optim.Adam(policy.parameters(), lr=1e-3)
optimizer =  optim.SGD(policy.parameters(), lr=1e-3,momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)
eval_epochs = 5

train(policy, 
      train_epochs=1000, 
      train_loader=train_loader, 
      val_loader=val_loader,
      optimizer=optimizer,
      loss_criterion=loss_criterion,
      scheduler=scheduler,
      thrashold = 100
    )

100%|██████████| 1000/1000 [02:13<00:00,  7.51it/s, train=0.00316, validation=0.00419]

###############################################################################

Train Loss: 0.0031616638456894593
Validation Loss: 0.0041933813598006965
###############################################################################



## Testing

In [25]:
def test(policy, test_loader,loss_criterion):
    policy.eval()   
    loss = 0
    epoch_loss = 0
    
    for batch_idx, (data, target) in enumerate(test_loader):

        obs, expert_action = data.to(device), target.to(device)
        obs = obs.float()
        
        if policy.name == 'bco-cnn':
            obs = obs.unsqueeze(1)

        student_action = policy(obs)
        expert_action = expert_action.float()

        loss = loss_criterion(student_action, expert_action)

        if not loss.item() == torch.inf: 
            epoch_loss += loss.item()
        
        if torch.isnan(student_action).any(): 
            print('e successo')
            break
        
    t_loss = epoch_loss/(batch_idx+1)
    epoch_loss = 0
            
    
    print("Test Loss: {}".format(t_loss))

In [28]:
test(policy=policy, test_loader=test_loader, loss_criterion=loss_criterion)

Test Loss: 0.004024738422594964


In [37]:
for i in range(90,120):
  obs, action  = test_loader.dataset.__getitem__(i)
  print(f'real action: {action}')
  obs = obs.float().unsqueeze(0).to(device)
  if policy.name == 'bco-cnn':
    obs = obs.unsqueeze(1)
  our_action = policy(obs).squeeze().detach().cpu().numpy()
  print(f'our action: {our_action}')

real action: tensor([ 0.2079,  0.0750, -0.0519,  0.3460, -0.2242, -0.0439,  0.0997,  0.3590,
        -0.0427, -0.0240, -0.2708,  0.4038,  0.4511, -0.4858,  0.0186,  0.0818,
         0.4763,  0.0567, -0.2460,  0.0600,  0.2073,  0.5226, -0.0345, -0.0079,
        -0.3282,  0.3981,  0.0718, -0.4590, -0.1065, -0.1380,  0.3512,  0.3494,
         0.2900, -0.1266,  0.1124,  0.2788])
our action: [ 0.24553178  0.0561863  -0.06466045  0.3505609  -0.29735693 -0.05971831
  0.11684342  0.36295122 -0.21373698  0.01865464 -0.27952805  0.35802728
  0.43054911 -0.37789407  0.03851409  0.00200479  0.47476834  0.1034334
 -0.21837084  0.03745013  0.18720102  0.5535536  -0.2623006   0.00449456
 -0.33830553  0.37798178  0.1501953  -0.4424153  -0.05636201 -0.15485153
  0.3105766   0.20216343  0.29765055 -0.11444138  0.10759443  0.41939908]
real action: tensor([-3.4693e-01,  2.0794e-02, -1.8087e-02,  3.7104e-01, -5.4264e-02,
        -4.2469e-02,  2.9747e-02,  4.8131e-01,  1.5012e+00, -1.7926e-03,
         1.09

In [ ]:
# not normalized observations / manual normalization
stop_at = 5
policy.eval()
for obs in expert_observations:
  print(obs)
  inputs = torch.from_numpy(obs[:196]).float().unsqueeze(0)
  inputs = 2 * ((inputs - min_val) / (max_val - min_val)) - 1
  print(inputs)
  output = policy(inputs).squeeze().detach().cpu().numpy()
  # print(f'our action: {our_action}')

In [ ]:
# Normalized observations
for i in range(80,110):
  obs = train_loader.dataset.__getitem__(i)[0]
  obs = obs.float().unsqueeze(0).to(device)
  print(obs)


## Saving

In [39]:
# Save model

# save_parameters()
version = 1
policy.save_parameters(root_dir+'/checkpoints/', version)

## Loading 

In [37]:
src = root_dir+'/checkpoints/'+policy.name.lower()+'.pt'
policy.load_parameters(src)
print(root_dir+'/checkpoints/'+policy.name.lower()+'.pt')

Loading model Behavioral-Cloning-Agent state parameters
From :/home/bitfra/Desktop/Francesco/rl_project/acrobatic-agents/checkpoints/behavioral-cloning-agent.pt
/home/bitfra/Desktop/Francesco/rl_project/acrobatic-agents/checkpoints/behavioral-cloning-agent.pt


# ENCODER

## Model

## Training

## Testing

## Saving